# HIPAA Compliance & PII Masking

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/04_compliance_governance/hipaa_gdpr_pii_masking/tutorial_hipaa_compliance.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/04_compliance_governance/hipaa_gdpr_pii_masking/tutorial_hipaa_compliance.ipynb)

## Business Scenario

A healthcare provider is ingesting patient medical records into their Data Lakehouse.
These records contain **Protected Health Information (PHI)** and **Personally Identifiable
Information (PII)** — patient names, Social Security Numbers, and email addresses.

The organization has two strict requirements:
1. **Governance Consistency** — every medical dataset must adhere to the institutional HIPAA
   policy (SSN formats, email validation) without re-specifying rules in each contract.
2. **Risk Mitigation** — all PII must be masked automatically before it reaches the Silver
   layer, so data scientists can work with the data without exposure to raw identifiers.

## Value Proposition

- **Zero-Day Compliance** — apply shared HIPAA Policy Packs across hundreds of tables instantly
- **Automated Protection** — shift-left security: PII is masked at ingestion, not as an afterthought
- **Audit-Ready Quarantine** — every failed record is tagged with lineage and the exact rule violation

---

## Goals

1. Set up paths to the contract, HIPAA Policy Pack, and source data
2. Inspect the HIPAA Policy Pack rules
3. Run ingestion with PII masking
4. Inspect quarantined rows (governance failures)
5. Inspect the masked Silver layer (safe for analytics)


## Setup

In [ ]:
import importlib.util
import os
import sys
from pathlib import Path

if importlib.util.find_spec("lakelogic") is None:
    import subprocess

    subprocess.run([sys.executable, "-m", "pip", "install", "lakelogic", "-q"], check=True)
    print("lakelogic installed.")
else:
    print("lakelogic ready.")

if "google.colab" in sys.modules:
    repo = Path("/content/LakeLogic")
    if not repo.exists():
        import subprocess

        subprocess.run(
            [
                "git",
                "clone",
                "--quiet",
                "https://github.com/lakelogic/LakeLogic.git",
                str(repo),
            ],
            check=True,
        )
    os.chdir(repo / "examples" / "04_compliance_governance" / "hipaa_gdpr_pii_masking")
    print(f"Working directory: {Path.cwd()}")


def resolve_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        for candidate in [
            base / name,
            base / "examples" / "04_compliance_governance" / name,
        ]:
            if candidate.exists():
                return candidate
    return cwd / name


def resolve_repo_root(start: Path) -> Path:
    for base in [start] + list(start.parents):
        if (base / "policy_packs" / "hipaa_compliance.yaml").exists():
            return base
    return start


base_path = resolve_example_dir("hipaa_gdpr_pii_masking")
repo_root = resolve_repo_root(base_path)
contract_path = base_path / "contracts" / "medical_records.yaml"
policy_pack_path = repo_root / "policy_packs" / "hipaa_compliance.yaml"
data_path = base_path / "data" / "phi_records.csv"

print(f"Contract   : {contract_path}")
print(f"Policy Pack: {policy_pack_path}")
print(f"Data       : {data_path}")
print("Setup complete.")

## How It Works

LakeLogic applies two layers of protection in a single pass:

```
phi_records.csv  (raw PHI: names, SSNs, emails)
      |
      v  DataProcessor.run_source()
      |    Layer 1: Quality gates from HIPAA Policy Pack
      |             - SSN format (XXX-XX-XXXX)
      |             - Email validation (must contain @)
      |
      |    Layer 2: PII masking hook
      |             - patient_name → [PROTECTED]
      |             - ssn         → [PROTECTED]
      |             - email       → [PROTECTED]
      |
      ├── result.good  →  Silver layer (masked, safe for analytics)
      └── result.bad   →  Quarantine  (failed HIPAA rules + reason)
```

### HIPAA Policy Pack rules

| Rule | What it enforces | HIPAA relevance |
|---|---|---|
| SSN format | `XXX-XX-XXXX` pattern | De-identification standard |
| Email validation | Must contain `@` | Data accuracy requirement |

### Known source data

| Patient | SSN | Email | Expected outcome |
|---|---|---|---|
| Alice | 123-45-6789 | alice@example.com | ✅ Valid — PII masked in Silver |
| Bob | 111-22-3333 | bob@wrong-format | ❌ Quarantined — failed email rule |
| Charlie | 222-33-4444 | charlie@example.com | ✅ Valid — PII masked in Silver |
| Diane | 333-44-5555 | diane@example.com | ✅ Valid — PII masked in Silver |
| Edward | 555-555-555 | edward@example.com | ❌ Quarantined — malformed SSN |


## 1. Load and Inspect the HIPAA Policy Pack

A **Policy Pack** acts as an organisational baseline. The HIPAA pack enforces rules across
every contract that references it — the compliance team manages it once; data engineers just point to it.


In [ ]:
import yaml

with open(policy_pack_path, "r") as f:
    policy_pack = yaml.safe_load(f)

print("HIPAA Policy Pack rules:")
for rule in policy_pack.get("quality", {}).get("row_rules", []):
    print(f"  - {rule.get('name')}: {rule.get('description')}")

## 2. Run Ingestion with PII Masking

Pass the contract to `DataProcessor`. The contract references the HIPAA Policy Pack and
activates the `pii_masking` hook — both are applied automatically during `run_source()`.


In [ ]:
from lakelogic import DataProcessor

processor = DataProcessor(contract=contract_path)
result = processor.run_source(data_path)

print(f"Source records : {len(result.raw)}")
print(f"Valid (Silver) : {len(result.good)}")
print(f"Quarantined    : {len(result.bad)}")
print(f"Reconciled     : {len(result.raw)} = {len(result.good)} + {len(result.bad)}")

## 3. Inspect the Quarantine (Governance Failures)

Bob (invalid email) and Edward (malformed SSN) were caught by the shared HIPAA rules
and diverted to quarantine — with the exact failure reason attached for compliance audit.


In [ ]:
print("QUARANTINE — Failed HIPAA Rules:")
display(result.bad)

## 4. Inspect the Masked Silver Layer

For the valid records (Alice, Charlie, Diane), `patient_name`, `ssn`, and `email`
have been replaced with `[PROTECTED]`. This data is safe for analytics teams — no raw PHI exposure.


In [ ]:
print("SILVER LAYER — Clean & Masked (HIPAA Safe):")
display(result.good)

---

## Summary

| Record | Outcome | Reason |
|---|---|---|
| Alice | ✅ Silver (masked) | Passes all HIPAA rules |
| Bob | ❌ Quarantined | Failed email validation |
| Charlie | ✅ Silver (masked) | Passes all HIPAA rules |
| Diane | ✅ Silver (masked) | Passes all HIPAA rules |
| Edward | ❌ Quarantined | Failed SSN format check |

### What LakeLogic did automatically

- Applied the HIPAA Policy Pack rules to every row — no per-contract repetition
- Executed the `pii_masking` hook in-flight during ingestion
- Replaced `patient_name`, `ssn`, and `email` in `result.good` with `[PROTECTED]`
- Quarantined failing rows to `result.bad` with the exact rule violation reason
- Added `_lakelogic_processed_at` and `_lakelogic_run_id` audit columns

---

## Next Steps — Try It Yourself

### 1. Edit the source data

Open `data/phi_records.csv` and add a row:

```csv
patient_id,patient_name,ssn,email,diagnosis
P006,Frank,999-88-7777,frank-no-at-sign,Flu
```

Re-run — Frank should land in quarantine (invalid email format).

### 2. Edit the contract

```yaml
# contracts/medical_records.yaml
policy_pack: hipaa_compliance       # references the shared HIPAA rules

hooks:
  post_validate:
    - name: pii_masking
      columns: [patient_name, ssn, email]   # <-- add or remove columns here
      replacement: "[PROTECTED]"
```

**Key contract knobs:**

| What to change | Where | Effect |
|---|---|---|
| Columns to mask | `hooks.post_validate[pii_masking].columns` | Add any PHI field |
| Mask replacement string | `hooks.post_validate[pii_masking].replacement` | Defaults to `[PROTECTED]` |
| Policy Pack used | `policy_pack` | Swap to `gdpr_compliance` for EU data |

### 3. Explore related tutorials

- [`tutorial_hipaa_gdpr_compliance.ipynb`](tutorial_hipaa_gdpr_compliance.ipynb) — HIPAA + GDPR side-by-side comparison
- [`../../02_core_patterns/bronze_quality_gate/`](../../02_core_patterns/bronze_quality_gate/playbook.ipynb) — quality gating without PII concerns
